<div style="text-align:center; cursor: auto;">
    <a href="https://www.credly.com/badges/5277e6b4-acf1-4f18-b83f-05d1d2ef3059/public_url" target="_blank">
        <img 
            src="applied-data-science-capstone.png" 
            width="150" 
            alt="IBM Applied Data Science Badge" 
            style="object-fit: cover; border-radius: 50%;">
    </a>
</div>

**<center><h2>Locations and Mapping with Folium</h2></center>**

## Launch Sites Locations Analysis with Folium


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


We will accomplish the following in this document:

*   Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*   Calculate the distances between a launch site to its proximities

Let's first import required Python packages for this lab:


In [2]:
import folium
import pandas as pd

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


First, let's add each site's location on a map using the sites' coordinates.


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
import io

spacex_csv_file = 'spacex_launch_geo.csv'
spacex_df = pd.read_csv(spacex_csv_file)

spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


We first need to create a folium `Map` object.


In [4]:
# NASA Johnson Space Center is the starting location
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

For more visual information, such as adding a highlighted circle area or text labels on specific coordinates, we use the `folium.Circle` class.

In [6]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In the map above, we can observe a small red circle near the city of Houston.

Now, let's add a circle for each launch site


In [7]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

launch_sites = {
    'CCAFS LC-40': [28.562302, -80.577356],
    'CCAFS SLC-40': [28.563197, -80.576820],
    'KSC LC-39A': [28.573255, -80.646895],
    'VAFB SLC-4E': [34.632834, -120.610745]
}

for site, coordinates in launch_sites.items():
    circle = folium.Circle(coordinates, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(site))
    marker = folium.map.Marker(
        coordinates,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site,
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


In [8]:
#creates MarkerCluster object
marker_cluster = MarkerCluster()

def assign_color(class_value):
    if class_value == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_color)

# adds marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location = [record['Lat'], record['Long']],
        icon = folium.Icon(color='white', icon_color = record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

Screenshots of updated map:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, we are able to easily identify which launch sites have relatively high success rates.


Next, we need to explore and analyze the proximities of launch sites.

In [9]:
# gets coordinates when hovering
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

We should now be able to zoom into our map and explore if there are any railways, highways, coastline, etc in the proximity.

In [10]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0 # approximate radius of earth in km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [11]:
# finds coordinate of closet coastline
launch_site_lat=28.56326
launch_site_lon=-80.57684
coastline_lat=28.56401
coastline_lon=-80.56811

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

# adds distance to coastline marker
distance_marker = folium.Marker(
   [coastline_lat,coastline_lon],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )
site_map.add_child(distance_marker)

# draws a line pointing to the closes coastline from each launch site
lines=folium.PolyLine(locations=[[launch_site_lat,launch_site_lon],[coastline_lat,coastline_lon]], weight=1)
site_map.add_child(lines)

site_map


Screenshots:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


#### Lastly, let's create markers for closest city, railway, highway, and coastlines.

In [12]:
coordinates = {
    'CCAFS LC-40': {'coastline': (28.5634,-80.56796), 'railway': (28.57217,-80.58528), 'highway': (28.56301,-80.57076), 'city': (28.10469,-80.64784)},
    'CCAFS SLC-40': {'coastline': (28.56409,-80.56806), 'railway': (28.57217,-80.58528), 'highway': (28.56385,-80.57088), 'city': (28.10469,-80.64784)},
    'KSC LC-39A': {'coastline': (28.60283,-80.58815), 'railway': (28.57314,-80.65398), 'highway': (28.57307,-80.65553), 'city': (28.10469,-80.64784)},
    'VAFB SLC-4E': {'coastline': (34.6347,-120.62531), 'railway': (34.63585,-120.62401), 'highway': (34.7048,-120.5694), 'city': (34.64253,-120.47331)}
}

proximities_df = spacex_df.groupby('Launch Site').first()[['Lat','Long']].assign(Proximity=lambda x:['self' for i in range(len(x))]).set_index('Proximity', append=True)

# updates dataframe with coordinates
for location, proximities in coordinates.items():
    for proximity, (lat, long) in proximities.items():
        proximities_df.loc[(location, proximity), ['Lat', 'Long']] = lat, long

# calculates distances
proximities_df = proximities_df.query("Proximity == 'self'").reset_index("Proximity",drop=True).join(
    proximities_df.query("Proximity != 'self'").reset_index("Proximity", drop=False),
    lsuffix="LaunchSite"
).assign(
    distance=lambda x:[calculate_distance(row['LatLaunchSite'], row['LongLaunchSite'], row['Lat'], row['Long']) for idx,row in x.iterrows()]
).set_index("Proximity", append=True)


proximities_df



LatLaunchSite  LongLaunchSite       Lat       Long  \
Launch Site  Proximity                                                       
CCAFS LC-40  coastline      28.562302      -80.577356  28.56340  -80.56796   
             railway        28.562302      -80.577356  28.57217  -80.58528   
             highway        28.562302      -80.577356  28.56301  -80.57076   
             city           28.562302      -80.577356  28.10469  -80.64784   
CCAFS SLC-40 coastline      28.563197      -80.576820  28.56409  -80.56806   
             railway        28.563197      -80.576820  28.57217  -80.58528   
             highway        28.563197      -80.576820  28.56385  -80.57088   
             city           28.563197      -80.576820  28.10469  -80.64784   
KSC LC-39A   coastline      28.573255      -80.646895  28.60283  -80.58815   
             railway        28.573255      -80.646895  28.57314  -80.65398   
             highway        28.573255      -80.646895  28.57307  -80.65553   
             city           28.573255      -80.646895  28.10469  -80.64784   
VAFB SLC-4E  coastline      34.632834     -120.610745  34.63470 -120.62531   
             railway        34.632834     -120.610745  34.63585 -120.62401   
             highway        34.632834     -120.610745  34.70480 -120.56940   
             city           34.632834     -120.610745  34.64253 -120.47331   

                         distance  
Launch Site  Proximity             
CCAFS LC-40  coastline   0.926054  
             railway     1.343093  
             highway     0.649222  
             city       51.365738  
CCAFS SLC-40 coastline   0.861525  
             railway     1.295798  
             highway     0.584818  
             city       51.471476  
KSC LC-39A   coastline   6.613767  
             railway     0.692172  
             highway     0.843713  
             city       52.118441  
VAFB SLC-4E  coastline   1.349004  
             railway     1.259455  
             highway     8.853368  
             city       12.623666

In [13]:
proximities_df.groupby(level='Proximity')['distance'].mean()

Proximity
city         41.894830
coastline     2.437587
highway       2.732780
railway       1.147629
Name: distance, dtype: float64

In [14]:
for idx,row in proximities_df.iterrows():
        
    distance_marker = folium.Marker(
        [row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 20; color:#d35400; text-transform: uppercase; font-weight:bold;"><b>%s, %s</b></div>' % (idx[1], "{:10.2f} km".format(row['distance'])),
        )
    )

    site_map.add_child(distance_marker)
    
    lines=folium.PolyLine(locations=[[row['LatLaunchSite'], row['LongLaunchSite']], [row['Lat'], row['Long']]], weight=1)
    site_map.add_child(lines)

site_map